In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

In [2]:
data = pd.read_parquet('./input/creditos_hist.parquet')

In [3]:
# Eliminamos las situaciones 0, que indican que el crédito ya fue pagado
data = data.loc[data['situacion'] != 0]

In [4]:
actividades = pd.read_parquet('./input/constancia_inscripcion.parquet')

In [5]:
actividades['idActividad'] = actividades['idActividad'].astype(str).str[:2]
actividades = actividades.pivot_table(
    index='identificacion',
    columns='idActividad',
    aggfunc='size',
    fill_value=0
)

columnas_actividades = [str(col) for col in actividades.columns]

actividades = actividades.astype(bool).astype(int).reset_index()

In [6]:
data = data.merge(actividades, on = 'identificacion', how= 'left')
del actividades

In [7]:
data = data.drop('denominacion', axis = 1)
data = data.dropna()

In [8]:
np.random.seed(42)
cuits = data['identificacion'].unique()
moneda = np.random.binomial(1, 0.25, len(cuits))
cuits_aleatorios = cuits[moneda == 1]

In [9]:
data = data.loc[data['identificacion'].isin(cuits_aleatorios)]

# Variables de interés
Creamos nuevas variables que creemos que pueden ser útiles para predecir el default

In [10]:
# Una variable que puede ser de interés es cuantos créditos tiene una empresa en un momento dado del tiempo
counts = data.groupby(['identificacion', 'periodo']).size().reset_index(name='n_creditos')

# También nos interesa cuanta plata debe una empresa en cada momento dado
sums = data.groupby(['identificacion', 'periodo'], as_index=True)['monto'].sum().reset_index(name='sum_montos')

# La literatura indica que también importa la duración de la relación empresa-banco, por lo que contamos la cantidad 
# de periodos que aparece cada par: empresa-banco
period_counts = data.groupby(['identificacion', 'entidad']).size().reset_index(name='n_periodos')

# Definimos como default cuando el crédito se encuentra en situación 4 o 5, por lo que creamos la dummy de default
# Esta es nuestra variable dependiente
data['default'] = (data['situacion'] > 4).astype(int)

In [11]:
# Agregamos las nuevas variables al dataframe
data = data.merge(counts, on=['identificacion', 'periodo'], how='left')
data = data.merge(sums, on=['identificacion', 'periodo'], how='left')
data = data.merge(period_counts, on=['identificacion', 'entidad'], how='left')

del sums, counts, period_counts

In [12]:
# Por último, la literatura también resalta que la intensidad de la relación empresa-banco es relevante
# Usamos como proxy para la intensidad la proporción del monto adeudado con un banco sobre el total adeudado
data['monto_relativo'] = data['monto'] / data['sum_montos']

In [13]:
# Ponemos bien el tipo de dato para las columnas categóricas, así el get_dummies funciona bien
data['identificacion'] = data['identificacion'].astype('category')
data['entidad'] = data['entidad'].astype('category')
data['situacion'] = data['situacion'].astype('category')
data['default'] = data['default'].astype('category')
data['periodo'] = data['periodo'].astype('category')
data[columnas_actividades] = data[columnas_actividades].astype(bool)

In [14]:
columnas = ['entidad', 'monto', 'n_creditos', 'sum_montos', 'n_periodos', 'monto_relativo']
columnas.extend(columnas_actividades)
X = pd.get_dummies(data[columnas])
Y = data['default']

In [15]:
sc = StandardScaler()
X = pd.DataFrame(sc.fit_transform(X), columns= X.columns)

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
del X, Y

También se puede hacer con neg_mean_squared_error, accuracy, f1_macro, f1_samples, average_precision, etc

In [17]:
model = LogisticRegressionCV(
    Cs=10, cv=5, penalty='l1', solver='saga', scoring='accuracy', n_jobs=-1, tol = 1e-3, max_iter=1000, random_state= 42, fit_intercept= True
)
model.fit(X_train, Y_train)

KeyboardInterrupt: 

In [ ]:
best_C = model.C_  # Accede al mejor valor de C
print(f"El mejor valor de C es: {best_C}")

In [114]:
y_pred = model.predict(X_test)
cm = confusion_matrix(Y_test, y_pred)
print(cm)

[[48581    31]
 [  371   163]]


In [115]:
precision = precision_score(Y_test, y_pred)
recall = recall_score(Y_test, y_pred)
f1 = f1_score(Y_test, y_pred)
accuracy = accuracy_score(Y_test, y_pred)

print(precision)
print(recall)
print(f1)
print(accuracy)

0.8402061855670103
0.3052434456928839
0.4478021978021978
0.9918202905628128


Si predecimos siempre negativo:
|       | PN    | PP    |
|-------|-------|-------|
| **TN** | 48612 | 0     |
| **TP** | 534  | 0     |

* La precision sería: 0/0
* El recall sería: 0
* El f1 sería: 0
* El accuracy sería: 0.9891344158

In [121]:
coeficientes = model.coef_[0]  # Coeficientes para la clase positiva
proporcion_ceros = (coeficientes == 0).sum() / len(coeficientes)
print(f"Proporción de coeficientes iguales a 0: {proporcion_ceros:.2%}")


Proporción de coeficientes iguales a 0: 13.62%
